In [1]:
import pandas as pd
import numpy as np

In [2]:
daily_basic = pd.read_csv('daily_basic.csv')
fina_indicator = pd.read_csv('fina_indicator.csv')
pro_bar = pd.read_csv('pro_bar.csv')

In [3]:
daily_basic.shape

(873815, 14)

In [4]:
daily_basic.head()

,ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_mv,circ_mv
0,000001.SZ,20190102,0.3141,0.7088,0.99,6.8048,6.4428,0.7330,1.4917,1.4012,1.48,1.48,1.577961e+07,1.577946e+07
1,000001.SZ,20190103,0.2420,0.5460,0.76,6.8714,6.5059,0.7402,1.5063,1.4149,1.47,1.47,1.593414e+07,1.593399e+07
2,000001.SZ,20190104,0.8626,1.9463,2.87,7.2194,6.7456,0.7607,1.5825,1.4343,1.39,1.39,1.674115e+07,1.674099e+07
3,000001.SZ,20190107,0.5042,1.1375,1.19,7.2120,6.7386,0.7599,1.5809,1.4329,1.40,1.40,1.672398e+07,1.672382e+07
4,000001.SZ,20190108,0.2344,0.5287,0.52,7.1528,6.6833,0.7536,1.5679,1.4211,1.41,1.41,1.658662e+07,1.658646e+07


In [5]:
fina_indicator.shape

(11190, 6)

In [6]:
fina_indicator.head()

,ts_code,ann_date,end_date,profit_dedt,roe,roa
0,000001.SZ,20191022.0,20190930,2.352700e+10,8.9454,NaN
1,000001.SZ,20190808.0,20190630,1.531600e+10,6.2028,NaN
2,000001.SZ,20190424.0,20190331,7.422000e+09,3.0331,NaN
3,000002.SZ,20191025.0,20190930,1.817556e+10,11.2933,2.9457
4,000002.SZ,20190821.0,20190630,1.174968e+10,7.4763,2.0049


In [7]:
pro_bar.shape

(875563, 11)

In [8]:
pro_bar.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20191227,16.53,16.93,16.43,16.63,16.47,0.16,0.9715,1042574.72,1741473.179
1,000001.SZ,20191226,16.34,16.48,16.32,16.47,16.30,0.17,1.0429,372033.86,610381.757
2,000001.SZ,20191225,16.45,16.56,16.24,16.30,16.40,-0.10,-0.6098,414917.98,679664.596
3,000001.SZ,20191224,16.23,16.50,16.23,16.40,16.24,0.16,0.9852,459128.42,752351.618
4,000001.SZ,20191223,16.68,16.68,16.17,16.24,16.59,-0.35,-2.1097,715792.72,1173372.080


In [9]:
# 将行情数据与财务数据结合
daily_basic_pro_bar = daily_basic.merge(pro_bar, left_on=['ts_code', 'trade_date'], right_on=['ts_code', 'trade_date'])
# 新股在merge中会丢失一部分财务信息
factor_model = daily_basic_pro_bar.merge(fina_indicator, left_on=['ts_code', 'trade_date'], right_on=['ts_code', 'ann_date'], how='left')
daily_basic_pro_bar.to_csv('daily_basic_w_pro_bar.csv', index=0)
factor_model.to_csv('factor_model.csv', index=0)

In [10]:
# 财务指标使用前值填充空缺
# 同一标的才能fillna！
col = fina_indicator.columns[3:]
factor_model[col] = factor_model.groupby('ts_code')[col].transform(lambda x: x.fillna(method='ffill', axis=0))
factor_model.to_csv('factor_model_fillna.csv', index=0)

In [11]:
factor_model

,ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,...,pre_close,change,pct_chg,vol,amount,ann_date,end_date,profit_dedt,roe,roa
0,000001.SZ,20190102,0.3141,0.7088,0.99,6.8048,6.4428,0.7330,1.4917,1.4012,...,9.2822,-0.1880,-2.0254,539386.32,498695.109,NaN,NaN,NaN,NaN,NaN
1,000001.SZ,20190103,0.2420,0.5460,0.76,6.8714,6.5059,0.7402,1.5063,1.4149,...,9.0942,0.0891,0.9797,415537.95,384457.707,NaN,NaN,NaN,NaN,NaN
2,000001.SZ,20190104,0.8626,1.9463,2.87,7.2194,6.7456,0.7607,1.5825,1.4343,...,9.1833,0.4651,5.0646,1481159.06,1422149.888,NaN,NaN,NaN,NaN,NaN
3,000001.SZ,20190107,0.5042,1.1375,1.19,7.2120,6.7386,0.7599,1.5809,1.4329,...,9.6484,-0.0099,-0.1026,865687.66,841166.430,NaN,NaN,NaN,NaN,NaN
4,000001.SZ,20190108,0.2344,0.5287,0.52,7.1528,6.6833,0.7536,1.5679,1.4211,...,9.6385,-0.0792,-0.8217,402388.11,389247.795,NaN,NaN,NaN,NaN,NaN
5,000001.SZ,20190109,0.7184,1.6208,1.67,7.3601,6.8770,0.7755,1.6134,1.4623,...,9.5593,0.2771,2.8987,1233486.36,1229465.116,NaN,NaN,NaN,NaN,NaN
6,000001.SZ,20190110,0.6242,1.4084,1.22,7.4786,6.9877,0.7880,1.6394,1.4858,...,9.8364,0.1583,1.6093,1071817.66,1079711.035,NaN,NaN,NaN,NaN,NaN
7,000001.SZ,20190111,0.4056,0.9150,0.69,7.5526,7.0569,0.7958,1.6556,1.5006,...,9.9947,0.0990,0.9905,696364.55,708001.802,NaN,NaN,NaN,NaN,NaN
8,000001.SZ,20190114,0.2915,0.6576,0.59,7.4860,6.9946,0.7887,1.6410,1.4873,...,10.0937,-0.0891,-0.8827,500443.59,507862.882,NaN,NaN,NaN,NaN,NaN
9,000001.SZ,20190115,0.3158,0.7124,0.69,7.5823,7.0846,0.7989,1.6621,1.5064,...,10.0046,0.1287,1.2864,542160.55,553027.309,NaN,NaN,NaN,NaN,NaN
